In [1]:
import sys
import os
import warnings
warnings.filterwarnings("ignore")

home_dir = "../"
src_path = os.path.join(home_dir, "src")

# Add the `src` folder to the Python path
sys.path.append(src_path)

from functions import (compute_distance_correlation_matrix,
 compute_simplicial_complex_and_landscapes, compute_wto_matrix, compute_pearson_correlation_matrix,
  patient_correlation_measure)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import time

In [2]:
# Load data from cancer stages
expression_matrix = pd.read_csv("../data/treatment_response/fpkm_matrix.csv", index_col=0)
significant_genes = pd.read_csv("../data/treatment_response/significant_genes.csv", index_col=0)

# Separate phenotype labels
phenotype = expression_matrix["phenotype"]
expression_matrix = expression_matrix.drop(columns=["phenotype"])

# Select significant genes
significant_gene_names = significant_genes.index
sig_exp_matrix = expression_matrix[significant_gene_names.intersection(expression_matrix.columns)]

top_var_genes_data = sig_exp_matrix
gene_dict = {i: col_name for i, col_name in enumerate(top_var_genes_data.columns)}
top_var_genes_data["phenotype"] = phenotype.values

resistant_top_var_genes_data = top_var_genes_data[top_var_genes_data['phenotype'] == 'Resistance']
sensitive_top_var_genes_data = top_var_genes_data[top_var_genes_data['phenotype'] == 'Sensitive']


resistant_top_var_genes_data = resistant_top_var_genes_data.drop(columns=["phenotype"])
sensitive_top_var_genes_data = sensitive_top_var_genes_data.drop(columns=["phenotype"])


# Split the data into training and testing sets for resistent
resistant_train, resistant_test = train_test_split(
    resistant_top_var_genes_data, test_size=0.2, random_state=42
)

# Split the data into training and testing sets for sensitive
sensitive_train, sensitive_test = train_test_split(
    sensitive_top_var_genes_data, test_size=0.2, random_state=42
)


resistant_distance_corr_matrix = compute_distance_correlation_matrix(resistant_train.values)
sensitive_distance_corr_matrix = compute_distance_correlation_matrix(sensitive_train.values)

# Define number of landscapes and resolution
num_landscape = 2
resolution = 100



In [3]:
start_time = time.time()
# Compute landscapes for training and testing sets for resistant
resistant_train_landscapes = compute_simplicial_complex_and_landscapes(
    resistant_train.values, resistant_distance_corr_matrix, num_landscape, resolution
)
resistant_test_landscapes = compute_simplicial_complex_and_landscapes(
    resistant_test.values, resistant_distance_corr_matrix, num_landscape, resolution
)

# Compute landscapes for training and testing sets for sensitive
sensitive_train_landscapes = compute_simplicial_complex_and_landscapes(
    sensitive_train.values, sensitive_distance_corr_matrix, num_landscape, resolution
)
sensitive_test_landscapes = compute_simplicial_complex_and_landscapes(
    sensitive_test.values, sensitive_distance_corr_matrix, num_landscape, resolution
)


# Combine training landscapes
train_landscapes = np.vstack([resistant_train_landscapes, sensitive_train_landscapes])
train_labels = np.concatenate([
    np.full(resistant_train_landscapes.shape[0], 1),  # Label 1 for resistant
    np.full(sensitive_train_landscapes.shape[0], 2) # Label 2 for sensitive
])

# Combine testing landscapes
test_landscapes = np.vstack([resistant_test_landscapes, sensitive_test_landscapes])
test_labels = np.concatenate([
    np.full(resistant_test_landscapes.shape[0], 1),  # Label 1 for stage 1
    np.full(sensitive_test_landscapes.shape[0], 2)
])

# Check shapes
print("Training Landscapes shape:", train_landscapes.shape)
print("Training Labels shape:", train_labels.shape)
print("Testing Landscapes shape:", test_landscapes.shape)
print("Testing Labels shape:", test_labels.shape)
end_time = time.time()

Computing the Simplicial Complex and persistence for patients


100%|██████████| 84/84 [00:01<00:00, 78.96it/s]


Empty list or empty diagrams: sample range is [-inf, -inf]
First value and second value in range are the same: grid is made of resolution copies of this value
Computing the Simplicial Complex and persistence for patients


100%|██████████| 22/22 [00:00<00:00, 85.72it/s]


Empty list or empty diagrams: sample range is [-inf, -inf]
First value and second value in range are the same: grid is made of resolution copies of this value
Computing the Simplicial Complex and persistence for patients


100%|██████████| 277/277 [00:03<00:00, 88.99it/s] 


Empty list or empty diagrams: sample range is [-inf, -inf]
First value and second value in range are the same: grid is made of resolution copies of this value
Computing the Simplicial Complex and persistence for patients


100%|██████████| 70/70 [00:00<00:00, 105.01it/s]

Empty list or empty diagrams: sample range is [-inf, -inf]
First value and second value in range are the same: grid is made of resolution copies of this value
Training Landscapes shape: (361, 600)
Training Labels shape: (361,)
Testing Landscapes shape: (92, 600)
Testing Labels shape: (92,)


In [4]:

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, mean_squared_error, log_loss
from sklearn.model_selection import GridSearchCV


# Define parameter grid for hyperparameter tuning
param_grid = {
    'n_estimators': [50, 100, 200],         # Number of trees in the forest
    'max_depth': [None, 10, 20],            # Maximum depth of the tree
    'min_samples_split': [2, 5, 10],        # Minimum samples to split an internal node
    'min_samples_leaf': [1, 2, 4], 
    'oob_score': [True]         # Minimum samples at a leaf node
}

# Perform Grid Search with Cross-Validation for Hyperparameter Tuning
grid_search = GridSearchCV(
    RandomForestClassifier(random_state=42),
    param_grid,
    cv=5,
    scoring='accuracy'
)
grid_search.fit(train_landscapes, train_labels)

# Get the best model from Grid Search
best_rf_model = grid_search.best_estimator_
print("Best parameters:", grid_search.best_params_)

# Train the best model on training data
best_rf_model.fit(train_landscapes, train_labels)

# Evaluate the best model
train_predictions = best_rf_model.predict(train_landscapes)
test_predictions = best_rf_model.predict(test_landscapes)
test_probabilities = best_rf_model.predict_proba(test_landscapes)

train_accuracy = accuracy_score(train_labels, train_predictions)
test_accuracy = accuracy_score(test_labels, test_predictions)
classification_rep = classification_report(test_labels, test_predictions)
conf_matrix = confusion_matrix(test_labels, test_predictions)

# Calculate metrics
mse = mean_squared_error(test_labels, np.argmax(test_probabilities, axis=1))
logloss = log_loss(test_labels, test_probabilities)

# Display results
print(f"Training Accuracy: {train_accuracy:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")
print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"Log Loss: {logloss:.4f}")
print("\nClassification Report:\n", classification_rep)
print("\nConfusion Matrix:\n", conf_matrix)
elapsed_time = (end_time - start_time) / 60
print(f"Elapsed time: {elapsed_time:.2f} seconds")

# Display OOB score
if hasattr(best_rf_model, 'oob_score_') and best_rf_model.oob_score_:
    print(f"OOB Score: {best_rf_model.oob_score_:.4f}")

Best parameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 50, 'oob_score': True}
Training Accuracy: 1.0000
Test Accuracy: 1.0000
Mean Squared Error (MSE): 1.0000
Log Loss: 0.0382

Classification Report:
               precision    recall  f1-score   support

           1       1.00      1.00      1.00        22
           2       1.00      1.00      1.00        70

    accuracy                           1.00        92
   macro avg       1.00      1.00      1.00        92
weighted avg       1.00      1.00      1.00        92


Confusion Matrix:
 [[22  0]
 [ 0 70]]
Elapsed time: 0.09 seconds
OOB Score: 0.9972


In [5]:
# Extract cross-validation results
cv_results = grid_search.cv_results_

# Convert results to a DataFrame for easier inspection
cv_results_df = pd.DataFrame(cv_results)

# Select and display important columns
columns_to_display = [
    'param_n_estimators',
    'param_max_depth',
    'param_min_samples_split',
    'param_min_samples_leaf',
    'mean_test_score',  # Average CV accuracy score for each parameter combination
    'std_test_score',   # Standard deviation of the CV scores
    'split0_test_score', 
    'split1_test_score', 
    'split2_test_score', 
    'split3_test_score', 
    'split4_test_score',
     'rank_test_score'
]
# Filter the results and sort by rank_test_score
cv_results_summary = cv_results_df[columns_to_display]
cv_results_summary_sorted = cv_results_summary.sort_values(by='rank_test_score')

# Save all results to a CSV file
cv_results_summary_sorted.to_csv('results/dc_cv_results.csv', index=False)

top_10_results = cv_results_summary_sorted.head(10)

# Display the top 10 results in this notebook environment
print("Top 10 CV Results:")
print(top_10_results)

Top 10 CV Results:
    param_n_estimators param_max_depth  param_min_samples_split  \
0                   50            None                        2   
57                  50              20                        5   
56                 200              20                        2   
55                 100              20                        2   
54                  50              20                        2   
53                 200              10                       10   
52                 100              10                       10   
51                  50              10                       10   
58                 100              20                        5   
50                 200              10                        5   

    param_min_samples_leaf  mean_test_score  std_test_score  \
0                        1         0.997222        0.005556   
57                       1         0.997222        0.005556   
56                       1         0.997222        0.0